# InventoryItem

Schema for an *InventoryItem*. This is the object that is uploaded to the database, managed between projects, used, requested, deleted, etc. It can take form as either a tool (reusable) or a material (used then removed). These are determined in the sub-enum called *ItemType*. When an item type is decided, it is possible to provide specifics for the item. If the item is a tool, then it will receive a QR code that corresponds with the tools' *objectID* (UUID). Another note: *id*, *objectID*, and *qrCodeID* are all the same values. *serialID* is what the tool actually has on it's label usually by the manufacterer. This helps to differentiate identical tools. If you have questions about this object, write them below:

In [ ]:
struct InventoryItem: FirebaseIdentifiable, Identifiable, Codable {

    @DocumentID var id: String?     // @DocumentID is firebases handling of the document id.
    var objectID: String            # objectID is the same value as id, but not optional.
    var itemType: ItemType
    var name: String
    var nameTrimmed: String
    var additionalInfo: String
    var categoryID: String
    var currentLocation: Location
    var currentHolderID: String
    var currentHolderName: String
    var currentHolderProfileImageURL: String
    var dateAdded: Date
    var isAvailable: Bool
    var isExpired: Bool
    var uploadedBy: String
    var imagesUrls: [String]
    var uiImages: [CodableImage] // for local use only
    var projectID: String
    var projectName: String

    // Add number of times checked in for seach algorithm purposes
    // Add CO2 kgs taken to make the item - This could be both attached to this item and categories

    // Specific properties
    var materialSpecifics: MaterialSpecifics?
    var toolSpecifics: ToolSpecifics?

    struct MaterialSpecifics: Codable, Hashable {
        var quantity: Int
        var specifications: [ItemSpecification]?
    }

    struct ToolSpecifics: Codable, Hashable {
        var qrCodeID: String
        var serialID: String
        let scanHistory: [String]?
        var brand: String?
    }
}

# Project

Below is the schema for a project within cambitas. This one is pretty straight forward. If you have any questions write them below:

In [ ]:
struct Project: FirebaseIdentifiable, Identifiable {

    @DocumentID var id: String?
    var objectID: String
    var companyCode: String
    var name: String
    var description: String?
    var timeCreated: Date
    var createdBy: String
    var startDate: Date
    var endDate: Date
    var location: Location
    var address: String
    var managerName: String
    var managerID: String
    var workersUserIDs: [String]?
    var inventoryItemIDs: [String]?
    var imagesUrls: [String]?

}

# User

Below is the object schema for an app user. If you have any questions please write them below:

In [ ]:
struct AppUser: Codable, Identifiable, Hashable {
    var id: String { uid }
    let uid: String
    var companyName: String
    var companyCode: String
    let email: String
    let displayName: String
    var profileImageUrl: String
    var mobileNo: String
    var role: Role = .admin
    let createdAt: Date
    var addedItemsID: [String]
    var scannedItemIDs: [String]
    let involvedProjectIDs: [String]

    enum Role: String, CaseIterable, Codable {
        case member = "Member"
        case manager = "Manager"
        case admin = "Admin"
    }
}

# Scan

Below is the object schema for a *Scan*. Scans will be used when the user checks an item in. So, every time the user either scans a QR code, or scans it in through an approved request, a scan object will be created in the database to track all the necessary information. If you have any questions please ask below:

In [ ]:
struct Scan: Codable, Identifiable, FirebaseIdentifiable {
    @DocumentID var id: String?
    var objectID: String
    let itemName: String
    let itemID: String
    let itemCategoryID: String
    let userID: String
    let serialID: String?
    let location: Location
    let timeScanned: Date
    let projectID: String
    let projectName: String
    let projectLocation: Location
}

# Company

This is the schema for the Company object. This object may need additions, so please let me know.
Any questions below:

In [ ]:
struct Company: Codable, Identifiable, Hashable {
    var id: String { companyCode }
    let companyName: String
    let companyCode: String
    let companyImageUrl: String?
    let createdAt: Date
    let websiteUrl: String?
    let address: String?
}

# Category

This is the current category management system I have put in place. This system was recommended by my uncle, who has been a professional software developer for over 15 years now. Here is how the system works:

### **IDs**
- ID's are of type String
- Each level of category is seperated by a colon ":".
- instead of having a subcategory as a *Category* object inside of itself, a subcategory is just another colon added to the id.
- for example, "5" would be a top level category, "5:2" would be a subcategory, and "5:2:6" would be a sub-sub-category.

#### **This ID system has pros and cons.**

Pros:
- allows for flat querying, all in one collection
- Much easier on the database, less querying
- able to retreive any layer of categories without retreiving any sub-categories.
- Able to query a layer of categories (as in, all top level categoies, or just all subcategories, etc) much easier.
- able to back track and see the top level category of a subcatgory easially.

Cons:
- A bit harder to implement, and slightly less intuitive

### **SpecificationSchema**

Other part of the *Category* object that is a bit complex is the *SpecificationSchema*. When adding a tool (reusable) to the database, we want to provide an option for the user to input "specifications" about the item. For example, if they are adding "steel", they could be prompted with the types fo steel, or the dimensions, etc.

- Within the Specification Schema, there is a SpecificationTypeSchema which determines what type of input the schema is expecting. Either being
  - single value
  - multiple values
  - optioned value (drop down menu)
  - string value (manual for user input)

- There is also a corresponding ItemSpecification object that uses a specification ID that takes the user input. Essentially, the Specification Schema provides the prompts for the user to input, while the ItemSpecification holds the actual user input.
- Why was it done this way? You may ask. Well, as it may be a bit complex, it is the cleanest, least memory and query intensive method that I have found.

If you have any questions or comments please leave them below:




In [ ]:
struct Category: Codable, Hashable, Identifiable {
    var id: String
    var name: String
    var selectable: Bool?
    var image: String?
    var specificationsSchemas: [SpecificationSchema]?
    func depth() -> Int {
        return id.components(separatedBy: ":").count
    }
}

# An example of a category object initialization is below. The top level category being "Materials", and the subcategory being "Steels"

    // MARK: - Materials
    Category(id: "1", name: "Materials", image: "materials.icon", specificationsSchemas: nil),

    // MARK: - Steels
    Category(id: "1:1", name: "Steels", specificationsSchemas: [
        SpecificationSchema(title: "Dimensions", type: .multipleValues(units: ["m", "cm"], valueTitles: ["Width", "Height", "Length"])),
        SpecificationSchema(title: "Steel type", type: .optionedValue(options: ["Alloy", "Stainless", "Tool", "Carbon", "Other"])),
        SpecificationSchema(title: "Grade", type: .optionedValue(options: ["304", "316", "A36", "O1", "Other"])),
        SpecificationSchema(title: "Finish", type: .optionedValue(options: ["Hot Rolled", "Cold Rolled", "Galvanized", "Other"])),
        SpecificationSchema(title: "Tensile Strength", type: .singleValue(units: ["MPa"])),
        SpecificationSchema(title: "Yield Strength", type: .singleValue(units: ["MPa"])),
        SpecificationSchema(title: "Elongation", type: .singleValue(units: ["%"])),
        SpecificationSchema(title: "Hardness", type: .singleValue(units: ["Rockwell"])),
        SpecificationSchema(title: "Heat Treatment", type: .optionedValue(options: ["Annealed", "Quenched", "Tempered", "Other"])),
        SpecificationSchema(title: "Coatings", type: .optionedValue(options: ["Zinc", "Nickel", "None", "Other"])),
        SpecificationSchema(title: "Application", type: .optionedValue(options: ["Construction", "Tools", "Machinery", "Other"]))
    ]),

In [ ]:
// MARK: - Spec Schema
struct SpecificationSchema: Identifiable, Codable, Hashable {
    var id = UUID()
    var title: String
    var type: SpecificationTypeSchema
}

enum SpecificationTypeSchema: Codable, Equatable, Hashable {
    case singleValue(units: [String])
    case multipleValues(units: [String], valueTitles: [String])
    case optionedValue(options: [String])
    case stringValue
}

// MARK: - User input
struct ItemSpecification: Identifiable, Codable, Hashable {
    var id = UUID()
    var schemaID: UUID // This references the Schema/Template
    var title: String
    var value: SpecificationValue


    func dictionary() -> [String: Any] {
        var dict: [String: Any] = [:]

        dict["id"] = self.id
        dict["schemaID"] = self.schemaID
        dict["title"] = self.title
        dict["value"] = self.value.dictionary()

        return dict
    }
}

enum SpecificationValue: Codable, Equatable, Hashable {
    case singleValue(selectedUnit: String?, value: Float?)
    case multipleValues(selectedUnit: String?, values: [Float?])
    case optionedValue(selectedOption: String?, customValue: String?)
    case stringValue(value: String?)

    func dictionary() -> [String: Any] {
        var dict: [String: Any] = [:]

        switch self {
        case .singleValue(let selectedUnit, let value):
            dict["selectedUnit"] = selectedUnit
            dict["value"] = value
        case .multipleValues(let selectedUnit, let values):
            dict["selectedUnit"] = selectedUnit
            dict["values"] = values
        case .optionedValue(let selectedOption, let customValue):
            dict["selectedOption"] = selectedOption
            dict["customValue"] = customValue
        case .stringValue(let value):
            dict["value"] = value
        }

        return dict
    }

}

# Request

Below is the Request object schema. This the what is created whenever one user needs an item from another, or there needs to be a transer of items from one project to another. This schema should be pretty straight forward. If you have any questions let me know below:

In [ ]:
struct Request: FirebaseIdentifiable, Identifiable {
    @DocumentID var id: String?
    var objectID: String
    var requesterID: String
    var requesterDisplayName: String
    var requesterEmail: String
    var requesterProfileImageURL: String
    var recipientID: String
    var recipientDisplayName: String
    var recipientEmail: String
    var recipientProfileImageURL: String
    var itemID: String
    var itemName: String
    var itemImageURLs: [String]
    var currentProjectID: String
    var currentProjectName: String
    var createdAt: Date
    var requestStatus: RequestStatus

    enum RequestStatus: String, CaseIterable, Codable {
        case pending = "Request Pending"
        case approved = "Request Accepted"
        case denied = "Declined"
        case checkedIn = "Item Checked In"
    }
}